In [1]:
import os
import numpy as np
import pandas as pd

## Sampling data

In [5]:
data_root = './data'

Under `data_root`, there are subset indices `train_idx.npy` and `test_idx.npy` for creating sample data.

In [2]:
sample_data_dir = './data/sample'

if not os.path.exists(sample_data_dir):
    os.makedirs(sample_data_dir)


In [8]:
train_idx = np.load(os.path.join(data_root, 'train_idx.npy'))
test_idx = np.load(os.path.join(data_root, 'test_idx.npy'))

tables = ['Demographics.csv', 'ICD.csv', 'INP_MED.csv', 'Labels.csv', 'LABS.csv', 'OUT_MED.csv']

x = pd.read_csv(os.path.join(data_root, tables[0]))

In [33]:
all_idx = x['idx'].unique()

not_select_train = list(set(all_idx.tolist()) - set(train_idx.tolist()))
not_select_test = list(set(all_idx.tolist()) - set(test_idx.tolist()))

select_map_train = dict(zip(train_idx.tolist(), [True for _ in range(len(train_idx))]))
not_select_map_train = dict(zip(not_select_train, [False for _ in range(len(not_select_train))]))

select_map_test = dict(zip(test_idx.tolist(), [True for _ in range(len(test_idx))]))
not_select_map_test = dict(zip(not_select_test, [False for _ in range(len(not_select_test))]))


In [34]:
for tb_name in tables:
    tb = pd.read_csv(os.path.join(data_root, tb_name))
    if 'Unnamed: 0' in tb.columns:
        tb = tb.drop('Unnamed: 0', axis=1)
    tb_sample_train = tb[tb['idx'].map(select_map_train | not_select_map_train)]
    tb_sample_test = tb[tb['idx'].map(select_map_test | not_select_map_test)]
    
    tb_merged = pd.concat([tb_sample_train, tb_sample_test], axis=0)

    tb_merged.to_csv(os.path.join(sample_data_dir, tb_name), index=False)

In [36]:
import shutil

for img_name in (train_idx.tolist() + test_idx.tolist()):
    shutil.copy(
        os.path.join(data_root, f'{img_name}.npy'),
        sample_data_dir
    )

In [43]:
import pickle

fd = open('./series_list.pkl', 'rb')
series_list = pickle.load(fd)

sereis_list_smaple = []
for s in series_list:
    if s.study_num in (train_idx.tolist() + test_idx.tolist()):
        sereis_list_smaple.append(s)

In [63]:
fd = open(os.path.join(sample_data_dir, 'series_list.pkl'), 'wb')
pickle.dump(sereis_list_smaple, fd)
fd.close()
